In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset,TensorDataset

## Only Seq2Seq model block

In [3]:
## Parameters
output_size = 1000
input_size = 1000

In [4]:
class Encoder(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size, embedding_size, p):
        super(Encoder,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.drop_out = nn.Dropout(p)
        self.embedding = nn.Embedding(embedding_dim = embedding_size, num_embeddings = input_size)
        self.lstm = nn.LSTM(embedding_size,hidden_size, num_layers, dropout=p, batch_first = True)
        
    def forward(self,x):
        
        # X shape = batch_size, sequence_length
        embedding = self.embedding(x)
        # embedding shape = batch_size, sequence_length, embedding_dimension/input_size
        outputs, (hidden, cell) = self.lstm(embedding)
        # output_dimension = num_layers, sequence_length, hidden_size 
        # hidden and cell state dimensions = num_layers, batch_size, hidden_size
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, embedding_size, num_layers, output_size, hidden_size, p): # output_size is vocab for the text to be translated
        super(Decoder,self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(num_embeddings=output_size, embedding_dim = embedding_size)
        """embedding_size = [batch_size, sequ_length, embedding_size]  but we want only one word to come first
            so the size should be [batch_size, 1, embedding_size]"""
        self.lstm = nn.LSTM(embedding_size, num_layers, hidden_size, dropout=p, batch_first = True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self,x, hidden, cell):
        x  = x.unsqueeze(1)
        # Embedding_size = batch_size,1,embedding_size
        embedding = self.dropout(self.embedding(x))
        
        #shape of output = batch_size, 1, hidden_size
        # shape of hidden and cell state num_layers,batch_size, hidden_size
        outputs, (hidden, cell) = self.lstm(embedding, (hidden, cell))
        # shape of predictions = batch_size, 1, length of vocab(output_size)
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(1)
        
        return predictions, hidden, cell
        

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, sos_idx, eos_idx, pad_idx):
        super(Seq2Seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx

    def forward(self, source, target, teacher_force_ratio = 0.5):
        batch_size = source.size(0)
        target_seq_len = target.size(1)
        output_size = self.decoder.fc.output_size
        outputs = torch.zeros(batch_size, target_seq_len, output_size)

        hidden, cell = self.encoder(source)

        # input = torch.full((batch_size,), self.sos_idx, dtype= torch.long)
        input = target[:,0]

        for t in range(1, target_seq_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:,t,:] = output

            # Decide whether to use teacher forcing or not
            teacher_force = torch.rand(1).item() < teacher_force_ratio

            # Get the predicted token for the next time step (greedy decoding)
            top1 = output.argmax(2)  # This is a tensor of shape (batch_size, 1)

            # If the predicted token is <EOS>, stop the sequence generation
            if top1 == self.eos_idx:
                break

            # Use teacher forcing or predicted token from the previous time step
            input = target[:, t] if teacher_force else top1
        return outputs

## Training Loop For Encoder Decoder

In [ ]:
# Training Hyperparameters
num_epochs = 20
learning_rate = 0.001
batch_size =32

# Model Hyperparameters
